# Course 1: Lidar
## Part 2: Point Cloud Segmentation 
#### By Jonathan L. Moran (jonathan.moran107@gmail.com)
From the Sensor Fusion Nanodegree programme offered at Udacity.

## Objectives

* Use the [Point Cloud Library](https://pointclouds.org/) (PCL) to segment point clouds;
* Implement the [Random Sample Consensus](https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://en.wikipedia.org/wiki/Random_sample_consensus) (RANSAC) algorithm for planar model fitting;
* Perform ground plane separation to "remove" the road surface from our point cloud data;

## 1. Introduction

### 1.1. Point Cloud Segmentation

#### Subsection

## 2. Programming Task

###

## 3. Closing Remarks

#### Alternatives


#### Extensions of task

## 4. Future Work

* ⬜️ ;
* ⬜️ .

## Credits

This assignment was prepared by Aaron Brown and Michael Maile of Mercedes-Benz Research & Development of North America (MBRDNA), 2021 (link [here](https://learn.udacity.com/nanodegrees/nd313/)).


References
* [1]

Helpful resources:
* 